<a href="https://colab.research.google.com/github/KennethTBarrett/DS-Unit-2-Linear-Models/blob/master/Copy_of_LS_DS_214_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [x] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [x] Begin with baselines for classification.
- [x] Use scikit-learn for logistic regression.
- [x] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [x] Get your model's test accuracy. (One time, at the end.)
- [x] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [x] Do one-hot encoding.
- [x] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# Setup Code

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [208]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


# Train / Validate / Test Split

In [209]:
# Let's see the beginning and end dates, may be able to just classify by year if it's simple enough.

df['Date'].sort_values(ascending = True) # Yup, simple enough!

305    1/10/2017
306    1/12/2017
307    1/12/2017
308    1/12/2017
309    1/12/2017
         ...    
223     9/5/2016
381     9/5/2017
382     9/5/2017
225     9/9/2016
224     9/9/2016
Name: Date, Length: 421, dtype: object

In [0]:
train = df[df['Date'].str.contains('^.*/.*/2016$')]
val = df[df['Date'].str.contains('^.*/.*/2017$')]
test = df[df['Date'].str.contains('^.*/.*/2018$')]

# Baselines for Classification


In [211]:
# Majority Class Baseline Whether the Burrito was Great (Majority Class)

target = 'Great'
y_train = train[target]
y_train.value_counts(normalize = True)

False    0.591216
True     0.408784
Name: Great, dtype: float64

In [212]:
# Training Accuracy

majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)

print('Baseline training accuracy if we guessed the majority class for each prediction:', accuracy_score(y_train, y_pred))

Baseline training accuracy if we guessed the majority class for each prediction: 0.5912162162162162


In [213]:
# Validation Accuracy

y_val = val[target]
y_pred = [majority_class] * len(y_val)

print('Baseline validation accuracy if we guessed the majority class for each prediction:', accuracy_score(y_val, y_pred))

Baseline validation accuracy if we guessed the majority class for each prediction: 0.5529411764705883


# Logistic Regression

## Features (Preparation)

### One-Hot Encoding

In [0]:
# Import Encoder
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names = True)

In [0]:
# Define High Cardinality Columns
high_cardinality = 'Date'

In [0]:
features = train.columns.drop([target] + [high_cardinality])
X_train = train[features]
X_test = test[features]
X_val = val[features]

In [0]:
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)
X_val = encoder.transform(X_val)

### Impute Missing Values

In [0]:
# Impute missing values
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
X_val_imputed = imputer.transform(X_val)

### Feature Scaling

In [0]:
# Import
from sklearn import preprocessing

In [0]:
X_train_scaled = preprocessing.scale(X_train_imputed)
X_val_scaled = preprocessing.scale(X_val_imputed)
X_test_scaled = preprocessing.scale(X_test_imputed)

y_test = test[target]

## Logistic Regression

In [0]:
# Import
from sklearn.linear_model import LogisticRegression

In [222]:
log_reg = LogisticRegression(solver = 'lbfgs')

log_reg.fit(X_train_scaled, y_train)
print('Training Accuracy (Logistic Regression):', log_reg.score(X_train_scaled, y_train))

log_reg.fit(X_val_scaled, y_val)
print('Validation Accuracy (Logistic Regression):', log_reg.score(X_val_scaled, y_val))

log_reg.fit(X_test_scaled, y_test)
print('Test Accuracy (Logistic Regression):', log_reg.score(X_test_scaled, y_test))

Training Accuracy (Logistic Regression): 0.9358108108108109
Validation Accuracy (Logistic Regression): 0.9647058823529412
Test Accuracy (Logistic Regression): 1.0
